In [126]:
import pandas as pd
import numpy as np
import re

In [127]:
#request = pd.read_csv('./Working_Data/COVIDDYNAMIC_Internal_Requests.csv', names=['label', 'details', 'extra'])
lookup = pd.read_excel('./Working_Data/Combined_Request_Lookup.xlsx', sheet_name='Qualt_lookup', index_col='Qualtrics_Request_Variable')
lookup_direc = pd.read_excel('./Working_Data/Combined_Request_Lookup.xlsx', sheet_name='Directory')
lookup_direc = lookup_direc.rename(columns=lookup_direc.iloc[0]).drop(lookup_direc.index[0])
lookup_direc.set_index('Request_Label', inplace=True)
data = pd.read_csv('./output/Wave1-17_A-N_release.csv', encoding = "ISO-8859-1", dtype=str)

In [128]:
lookup

,Group,Wave,Request_Label,Item
Qualtrics_Request_Variable,,,,
NaN,NaN,NaN,NaN,NaN
CVD_Data_1_1,CVD_Data_Cases,1.0,CVD_Data_Cases,Supplemental Data - COVID-19 cases & deaths by...
CVD_Data_2_1,CVD_Data_Cases,2.0,CVD_Data_Cases,Supplemental Data - COVID-19 cases & deaths by...
CVD_Data_3_1,CVD_Data_Cases,3.0,CVD_Data_Cases,Supplemental Data - COVID-19 cases & deaths by...
CVD_Data_4_1,CVD_Data_Cases,4 - A,CVD_Data_Cases,Supplemental Data - COVID-19 cases & deaths by...
...,...,...,...,...
GFPS36v2_2,Conte,E,GFPS36v2,"Protest Survey 2 - Wave 8 - June 27, 2020 (Sel..."
GFPS37v2_1,Prolific,8.0,GFPS37v2,"Protest Survey 2 - Wave 8 - June 27, 2020 (Sel..."
GFPS37v2_2,Conte,E,GFPS37v2,"Protest Survey 2 - Wave 8 - June 27, 2020 (Sel..."


In [132]:
lookup.loc['CVD_Consp_P_17']

Group                                                   Prolific
Wave                                                        17.0
Request_Label                                          CVD_Consp
Item             COVID_Conspiracies-Prolific-Wave 17-N (7/24/21)
Name: CVD_Consp_P_17, dtype: object

In [131]:
data.loc[data['wave']=='17', 'Cnsp1_1']

20031    2.0
20032    3.0
20033    1.0
20034    1.0
20035    4.0
        ... 
20665    3.0
20666    1.0
20667    3.0
20668    4.0
20669    4.0
Name: Cnsp1_1, Length: 639, dtype: object

In [228]:
all_conte_waves = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N']
all_prolific_waves = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '15b', '16', '17']
task_dir = r'C:\Users\andyl\Box\COVID-19 Adolphs Lab\PreProcessed_Data\tasks'

def extract_uvn(request_label, i):
    # get unified variable name(s)
    try:
        unified_variable_names = lookup_direc['unified_variable_names'].loc[request_label]
        if type(unified_variable_names) == str:
            unified_variable_names = [unified_variable_names]
        else:
            unified_variable_names = list(unified_variable_names)
        return unified_variable_names
    except KeyError:
        print('{:<30s}{:<30s}{:10s}'.format(str(i), str(request_label), 'x'))
        return None
    
    
def extract_data(request):
    variables_and_waves = {}
    print('{:<30s}{:<30s}{:10s}'.format('UNV', 'request_label', 'x'))
    print('========================================================')
    for i in request['label'].loc[5:]:
        try:
            wave = lookup['Wave'].loc[i]
            
            current_label = lookup['Request_Label'].loc[i]
            #print(current_label)
            #print(wave)
            if wave == 'All':
                #print(current_label)
                #print(lookup['Group'].loc[i])
                if lookup['Group'].loc[i] == 'Prolific':
                    selected_waves = all_prolific_waves.copy()
                    #print(selected_waves)
                if lookup['Group'].loc[i] == 'Conte':
                    selected_waves = all_conte_waves.copy()
                    #print(selected_waves)
                    #print(all_conte_waves)
                #print(selected_waves)
                

                UVN = extract_uvn(current_label, i)
                if UVN:
                    for var in UVN:
                        if var in variables_and_waves.keys():
                            variables_and_waves[var] = variables_and_waves[var] + selected_waves
                        else:
                            variables_and_waves[var] = selected_waves
            else:
                UVN = extract_uvn(current_label, i)
                if UVN:
                    for var in UVN:
                        if var in variables_and_waves.keys():
                            if type(wave) == int or type(wave) == float:
                                if str(int(wave)) not in variables_and_waves[var]:
                                    variables_and_waves[var].append(str(int(wave)))
                            else:
                                if str(wave) not in variables_and_waves[var]:
                                    variables_and_waves[var].append(str(wave))
                        else:
                            if type(wave) == int or type(wave) == float:
                                variables_and_waves[var] = [str(int(wave))]
                            else:
                                variables_and_waves[var] = [str(wave)]
        except (KeyError, ValueError):
            print('{:<30s}{:<30s}{:10s}{:10s}'.format(str(i), current_label, 'x', str(wave)))
            continue
    return variables_and_waves

def crop_data(request):
    variables_and_waves = extract_data(request)
    # Debug
    #print(variables_and_waves)
    variables_and_waves['V4'] = all_conte_waves + all_prolific_waves
    variables_and_waves['V3'] = all_conte_waves + all_prolific_waves

    # adding adminstration variables
    variables_and_waves['DO_Event_Surveys'] = all_conte_waves + all_prolific_waves
    variables_and_waves['DO_Main_Surveys'] = all_conte_waves + all_prolific_waves
    variables_and_waves['TASK1'] = all_conte_waves + all_prolific_waves
    variables_and_waves['TASK2'] = all_conte_waves + all_prolific_waves
    variables_and_waves['TASK3'] = all_conte_waves + all_prolific_waves
    variables_and_waves['TASK4'] = all_conte_waves + all_prolific_waves
    variables_and_waves['TASK5'] = all_conte_waves + all_prolific_waves
    variables_and_waves['RW27'] = all_conte_waves + all_prolific_waves
    
    # Adding quality measures
    quality_variables = lookup_direc.loc['Quality','unified_variable_names'].values
    for var in quality_variables:
        variables_and_waves[var] = all_conte_waves + all_prolific_waves
        
    columns = [col for col in variables_and_waves.keys() if col in data.columns]
    #print(columns)
    columns.extend(['wave', 'PROLIFIC_PID'])
    print('\nThese columns were not found in the data:')
    for i in [col for col in variables_and_waves.keys() if col not in data.columns]:
        print(i)

    output_data = data[columns]
    output_data['wave'] =  output_data['wave'].astype(str)
    output_data.set_index('wave', inplace=True)
    
    waves = list(set(list(output_data.index)))
    print(waves)
    for col in output_data.columns:
        if col == 'PROLIFIC_PID':
            continue
        to_nan = [w for w in waves if w not in variables_and_waves[col]]
        if col == 'Cnsp1_1':
            print(variables_and_waves[col])
        if col == 'Assist_EtOH_raw':
            print(variables_and_waves['Assist_EtOH_raw'][0])
            print(to_nan)
        output_data[col].loc[to_nan] = np.nan
    
    output_data.insert(0, 'wave', output_data.index)
    output_data.set_index('PROLIFIC_PID', inplace=True)
    
    return output_data


def clean_up(request):
    request = request.loc[request['extra'] == "1"]
    data = crop_data(request)
    columns = data.columns.tolist()
    columns.remove('wave')
    columns.remove('V4')
    subset = data.dropna(subset=columns, how='all')
    return subset

def inspect_request(filename):
    new_request = pd.read_csv(filename).T
    new_request = new_request.reset_index()
    new_request = new_request.iloc[17:]
    return new_request

def process_request(request_filename, request_col, save_filename):
    new_request = pd.read_csv(request_filename).T
    new_request = new_request.reset_index()
    new_request = new_request.iloc[17:]
    request = new_request.iloc[:, [0,1,request_col]].rename(columns={'index': 'label', 0: "details", request_col-1: "extra"})
    request = request.loc[request['extra'] == "1"]
    request.to_csv(save_filename)
    return request

def export_tasks(request, filename):
    task_cols = list(request.loc[request.label.str.startswith('Task').values, 'label'].values)
    
    # remove PGG from request for now
    for col_idx, i_col in enumerate(task_cols):
        if i_col.startswith('Task_C_PGG') | i_col.startswith('Task_P_PGG'):
            task_cols.pop(col_idx)
        
    filename = filename.strip('data.csv')
    filename = '.' + filename
    task = ''
    wave = []
    group = ''
    data = pd.DataFrame()
    for i_idx, i_task, in enumerate(task_cols):
        substr_idx = [m.start() for m in re.finditer('_', i_task)]
        if i_idx+1 < len(task_cols):
            i_task_1 = task_cols[i_idx+1]
            next_group = i_task_1[substr_idx[0]+1:substr_idx[1]]
            next_task = i_task_1[substr_idx[1]+1:substr_idx[2]]
        else:
            i_task_1 = ''
            next_group = ''
            next_task = ''
        curr_group = i_task[substr_idx[0]+1:substr_idx[1]]
        curr_task = i_task[substr_idx[1]+1:substr_idx[2]]
        curr_wave = i_task[substr_idx[2]+1:]
        if curr_wave == '99':
            task = curr_task
            group = curr_group
            data = pd.read_csv(task_dir + '\\' + task + '_' + group +'.csv')
            data.to_csv(filename + task + '_' + group + '.csv')
            data = pd.DataFrame()
        elif (task != curr_task) | (group != curr_group):
            task = curr_task
            group = curr_group
            tmp_data = pd.read_csv(task_dir + task + '_' + group +'.csv',index=False)
            wave_col = tmp_data.columns[tmp_data.columns.str.endswith('wave')].values[0]
            if group =='P':
                curr_wave = int(curr_wave)
            data = tmp_data.loc[tmp_data[wave_col] == curr_wave, :]
            data.to_csv(filename + task + '_' + group + '.csv',index=False)
        elif (task == curr_task) & (group == curr_group):
            wave_data = tmp_data.loc[tmp_data[wave_col] == int(curr_wave), :]
            data = data.append(wave_data)
            if (curr_task != next_task) | (curr_group != next_group):              
                data.to_csv(filename + task + '_' + group + '.csv', index=False)
                data = pd.DataFrame()
                

def export(request, filename, data):
    
    # Export task data
    export_tasks(request, filename)
    
    # Export main data
    data = crop_data(request)
    
    #print(data.loc[data['wave']=='17', 'Cnsp1_1'])
    
    columns = data.columns.tolist()
    # remove admin columns
    columns.remove('wave')
    columns.remove('V4')
    columns.remove('V3')
    columns.remove('DO_Event_Surveys')
    columns.remove('DO_Main_Surveys')
    columns.remove('TASK1')
    columns.remove('TASK2')
    columns.remove('TASK3')
    columns.remove('TASK4')
    columns.remove('TASK5')
    columns.remove('RW27')
    
    quality_variables = lookup_direc.loc['Quality','unified_variable_names'].values
    for var in quality_variables:
        try:
            columns.remove(var)
        except:
            print(var)
            continue

    subset = data.dropna(subset=columns, how='all')
    subset = subset.fillna('NA')
    subset.to_csv(filename)

In [183]:
box_path = r"C:\Users\andyl\Box\COVID-19 Adolphs Lab\approved_projects"

In [184]:
request_filename = r'C:\Users\andyl\Box\COVID-19 Adolphs Lab\approved_projects\Teresa_Lopez_Castro_A_year_like_no_other\request.csv'
request = inspect_request(request_filename)
request

,index,0,1
17,Invest_2,"Prior to submitting a data-request, the COVID-...",A year like no other: Longitudinal trajectorie...
18,Invest_3,"Prior to submitting a data-request, the COVID-...",Teresa Lopez-Castro
19,Invest_4,"Prior to submitting a data-request, the COVID-...",The City College of New York
20,Invest_5,"Prior to submitting a data-request, the COVID-...",lopezcastro.phd.ccny@gmail.com
21,Other_Investigators#1_1_1,All data will be distributed via Box to the PI...,NaN
...,...,...,...
1629,GFPS36v2_2,"Protest Survey 2 - Wave 8 - June 27, 2020 (Sel...",1
1630,GFPS37v2_1,"Protest Survey 2 - Wave 8 - June 27, 2020 (Sel...",1
1631,GFPS37v2_2,"Protest Survey 2 - Wave 8 - June 27, 2020 (Sel...",1
1632,GFPS38v2_1,"Protest Survey 2 - Wave 8 - June 27, 2020 (Sel...",1


In [229]:
# 1. Uri, Tom: Relation between Expressed Social Norms of COVID-19, Social Factors & Actual Preventative Behaviors
request_filename = r'C:\Users\andyl\Box\COVID-19 Adolphs Lab\approved_projects\Uri_Maoz_Tom_Relation_between_Expressed_Social_Norms\request.csv'
request_col = 2
save_filename = './output/approved_projects/Uri_Maoz_Tom_Relation_between_Expressed_Social_Norms/data.csv'
request = process_request(request_filename, request_col, save_filename)
export(request, save_filename, data)

UNV                           request_label                 x         
CVD_Data_1_3                  CVD_Data_Protest              x         
CVD_Data_1_4                  Unemployment_Stats            x         
CVD_Data_2_3                  CVD_Data_Protest              x         
CVD_Data_2_4                  Unemployment_Stats            x         
CVD_Data_3_3                  CVD_Data_Protest              x         
CVD_Data_3_4                  Unemployment_Stats            x         
CVD_Data_4_3                  CVD_Data_Protest              x         
CVD_Data_4_4                  Unemployment_Stats            x         
CVD_Data_5_3                  CVD_Data_Protest              x         
CVD_Data_5_4                  Unemployment_Stats            x         
CVD_Data_6_3                  CVD_Data_Protest              x         
CVD_Data_6_4                  Unemployment_Stats            x         
CVD_Data_7_3                  CVD_Data_Protest              x         
CVD_Da

C:\Users\andyl\anaconda3\envs\g\lib\site-packages\pandas\core\frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)
C:\Users\andyl\anaconda3\envs\g\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


['14', 'F', '8', '3', 'N', '15', 'K', '5', 'I', 'L', '13', 'M', 'J', '15b', 'E', 'C', '4', 'B', '7', 'G', 'A', '1', '9', '10', '16', 'D', '17', '11', '6', '12', '2', 'H']
8
['14', 'F', '3', 'N', '15', 'K', '5', 'I', 'L', 'M', 'J', '15b', 'E', 'C', '4', 'B', '7', 'G', 'A', '1', '9', '10', '16', 'D', '11', '6', '12', '2', 'H']
['17']


<ipython-input-229-a629d9d259f2>:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider using pd.concat instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  export(request, save_filename, data)


mismatch_dem_age


In [230]:
# 2. Teresa Lopez-Castro 5/24: A year like no other
request_filename = r'C:\Users\andyl\Box\COVID-19 Adolphs Lab\approved_projects\Teresa_Lopez_Castro_A_year_like_no_other\request.csv'
request_col = 2
save_filename = './output/approved_projects/Teresa_Lopez_Castro_A_year_like_no_other/data.csv'
request = process_request(request_filename, request_col, save_filename)
export(request, save_filename, data)

UNV                           request_label                 x         
LEC_C_2                       LEC                           x         17.0      
NIH_C_1                       NIH                           x         nan       
NIH_C_3                       NIH                           x         nan       
PSS_C_2                       PSS                           x         nan       
PSS_C_4                       PSS                           x         nan       

These columns were not found in the data:
state
cumulative_cases
new_cases
slope_new_cases
cumulative_deaths
new_deaths
slope_new_deaths
Mandatory_SAH
Recomm_SAH
Mandatory_gather_limit
Mandatory_gather_max
Recomm_gather_limit
Recomm_gather_max
GatheringStrictness
State_park_closure
Playground_closure
Mandatory_PPE_masks
Recomm_PPE_masks
Mandatory_business_closure
Recomm_business_closure
mismatch_dem_age
['14', 'F', '8', '3', 'N', '15', 'K', '5', 'I', 'L', '13', 'M', 'J', '15b', 'E', 'C', '4', 'B', '7', 'G', 'A', '1', 

<ipython-input-230-9c6b02db8b8d>:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider using pd.concat instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  export(request, save_filename, data)


mismatch_dem_age


In [231]:
# 3. Marina Weiss request 5/24: Longitudinal Impacts of Discrimination on Depressive Symptomatology in Racial/Ethnic Minorities During the COVID and Racial Injustice Pandemics
request_filename = r'C:\Users\andyl\Box\COVID-19 Adolphs Lab\approved_projects\Marina_Weiss_Longitudinal_Impacts_of_Discrimination\request.csv'
request_col = 2
save_filename = './output/approved_projects/Marina_Weiss_Longitudinal_Impacts_of_Discrimination/data.csv'
request = process_request(request_filename, request_col, save_filename)
export(request, save_filename, data)

UNV                           request_label                 x         
CVD_Data_1_3                  CVD_Data_Protest              x         
CVD_Data_1_4                  Unemployment_Stats            x         
CVD_Data_2_3                  CVD_Data_Protest              x         
CVD_Data_2_4                  Unemployment_Stats            x         
CVD_Data_3_3                  CVD_Data_Protest              x         
CVD_Data_3_4                  Unemployment_Stats            x         
CVD_Data_4_3                  CVD_Data_Protest              x         
CVD_Data_4_4                  Unemployment_Stats            x         
CVD_Data_5_3                  CVD_Data_Protest              x         
CVD_Data_5_4                  Unemployment_Stats            x         
CVD_Data_6_3                  CVD_Data_Protest              x         
CVD_Data_6_4                  Unemployment_Stats            x         
CVD_Data_7_3                  CVD_Data_Protest              x         
CVD_Da

<ipython-input-231-d030c309060f>:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider using pd.concat instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  export(request, save_filename, data)


mismatch_dem_age


In [232]:
# 4. Andy 5/5: Conspiracy Theory
request_filename = r'C:\Users\andyl\Box\COVID-19 Adolphs Lab\approved_projects\Andy_Liang_Uncovering_Heterogeneity_within_Conspiracy_Theorists\request.csv'
request_col = 2
save_filename = './output/approved_projects/Andy_Liang_Uncovering_Heterogeneity_within_Conspiracy_Theorists/data.csv'
request = process_request(request_filename, request_col, save_filename)
export(request, save_filename, data)


UNV                           request_label                 x         
Task_P_Consp_99               CONS_TSK                      x         

These columns were not found in the data:
mismatch_dem_age
['14', 'F', '8', '3', 'N', '15', 'K', '5', 'I', 'L', '13', 'M', 'J', '15b', 'E', 'C', '4', 'B', '7', 'G', 'A', '1', '9', '10', '16', 'D', '17', '11', '6', '12', '2', 'H']
8
['14', 'F', '3', 'N', '15', 'K', '5', 'I', 'L', 'M', 'J', '15b', 'E', 'C', '4', 'B', '7', 'G', 'A', '1', '9', '10', '16', 'D', '11', '6', '12', '2', 'H']
['17']


<ipython-input-232-18ace1d6e54e>:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider using pd.concat instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  export(request, save_filename, data)


mismatch_dem_age


In [233]:
request_filename = r'C:\Users\andyl\Box\COVID-19 Adolphs Lab\approved_projects\Andy_Liang_Uncovering_Heterogeneity_within_Conspiracy_Theorists\request.csv'
#request_filename = r'C:\Users\andyl\Box\COVID-19 Adolphs Lab\approved_projects\Nina_Memory\request.csv'
request = process_request(request_filename, request_col, save_filename)

In [234]:
request.loc[request['label']=='CVD_Consp_P_17']

,label,details,extra
308,CVD_Consp_P_17,COVID Conspiracies Questionnaire - Prolific - ...,1


In [235]:
# 5. Nina: Memory
request_filename = r'C:\Users\andyl\Box\COVID-19 Adolphs Lab\approved_projects\Nina_Memory\request.csv'
request_col = 2
save_filename = './output/approved_projects/Nina_Memory/data.csv'
request = process_request(request_filename, request_col, save_filename)
export(request, save_filename, data)


UNV                           request_label                 x         
CVD_Data_1_3                  CVD_Data_Protest              x         
CVD_Data_1_4                  Unemployment_Stats            x         
CVD_Data_2_3                  CVD_Data_Protest              x         
CVD_Data_2_4                  Unemployment_Stats            x         
CVD_Data_3_3                  CVD_Data_Protest              x         
CVD_Data_3_4                  Unemployment_Stats            x         
CVD_Data_4_3                  CVD_Data_Protest              x         
CVD_Data_4_4                  Unemployment_Stats            x         
CVD_Data_5_3                  CVD_Data_Protest              x         
CVD_Data_5_4                  Unemployment_Stats            x         
CVD_Data_6_3                  CVD_Data_Protest              x         
CVD_Data_6_4                  Unemployment_Stats            x         
CVD_Data_7_3                  CVD_Data_Protest              x         
CVD_Da

C:\Users\andyl\anaconda3\envs\g\lib\site-packages\pandas\core\frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)
C:\Users\andyl\anaconda3\envs\g\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


['14', 'F', '8', '3', 'N', '15', 'K', '5', 'I', 'L', '13', 'M', 'J', '15b', 'E', 'C', '4', 'B', '7', 'G', 'A', '1', '9', '10', '16', 'D', '17', '11', '6', '12', '2', 'H']
8
['14', 'F', '3', 'N', '15', 'K', '5', 'I', 'L', 'M', '15b', 'C', '4', 'B', '7', 'G', 'A', '1', '9', '10', '16', 'D', '17', '11', '6', '12', '2', 'H']


<ipython-input-235-0ba2f9b77d3a>:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider using pd.concat instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  export(request, save_filename, data)


mismatch_dem_age


In [236]:
# 6. Damian: EPII
request_filename = r'C:\Users\andyl\Box\COVID-19 Adolphs Lab\approved_projects\Damian_Stanley_EPII\request.csv'
request_col = 2
save_filename = './output/approved_projects/Damian_Stanley_EPII/data.csv'
request = process_request(request_filename, request_col, save_filename)
export(request, save_filename, data)

UNV                           request_label                 x         

These columns were not found in the data:
mismatch_dem_age
['14', 'F', '8', '3', 'N', '15', 'K', '5', 'I', 'L', '13', 'M', 'J', '15b', 'E', 'C', '4', 'B', '7', 'G', 'A', '1', '9', '10', '16', 'D', '17', '11', '6', '12', '2', 'H']
8
['14', 'F', '3', 'N', '15', 'K', '5', 'I', 'L', '13', 'M', 'J', '15b', 'E', 'C', '4', 'B', '7', 'G', 'A', '1', '9', '10', '16', 'D', '17', '11', '6', '12', '2', 'H']


<ipython-input-236-d5253ba05ca0>:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider using pd.concat instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  export(request, save_filename, data)


mismatch_dem_age


In [237]:
# 7. Teresa: Longitudinal_Trajectories_and_Predictors_of_Substance_Use_during_COVID
request_filename = r'C:\Users\andyl\Box\COVID-19 Adolphs Lab\approved_projects\Teresa_Lopez_Castro_Longitudinal_Trajectories_and_Predictors_of_Substance_Use_during_COVID\request.csv'
request_col = 2
save_filename = './output/approved_projects/Teresa_Lopez_Castro_Longitudinal_Trajectories_and_Predictors_of_Substance_Use_during_COVID/data.csv'
request = process_request(request_filename, request_col, save_filename)
export(request, save_filename, data)

UNV                           request_label                 x         

These columns were not found in the data:
mismatch_dem_age
['14', 'F', '8', '3', 'N', '15', 'K', '5', 'I', 'L', '13', 'M', 'J', '15b', 'E', 'C', '4', 'B', '7', 'G', 'A', '1', '9', '10', '16', 'D', '17', '11', '6', '12', '2', 'H']
8
['14', 'F', '3', 'N', '15', 'K', '5', 'I', 'L', 'M', 'J', '15b', 'E', 'C', '4', 'B', '7', 'G', 'A', '1', '9', '10', '16', 'D', '11', '6', '12', '2', 'H']


<ipython-input-237-d8a33767062b>:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider using pd.concat instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  export(request, save_filename, data)


mismatch_dem_age


In [238]:
# 8. Damian: Investigating the Impact of COVID-19 and the Events of 2020 on Race Attitudes in the United  States
request_filename = r'C:\Users\andyl\Box\COVID-19 Adolphs Lab\approved_projects\Damian_Stanley_Impact_of_COVID-19_and_the_Events_of_2020_on_Race_Attitudes\request.csv'
request_col = 2
save_filename = './output/approved_projects/Damian_Stanley_Impact_of_COVID-19_and_the_Events_of_2020_on_Race_Attitudes/data.csv'
request = process_request(request_filename, request_col, save_filename)
export(request, save_filename, data)

UNV                           request_label                 x         
CVD_Data_1_3                  CVD_Data_Protest              x         
CVD_Data_2_3                  CVD_Data_Protest              x         
CVD_Data_3_3                  CVD_Data_Protest              x         
CVD_Data_4_3                  CVD_Data_Protest              x         
CVD_Data_5_3                  CVD_Data_Protest              x         
CVD_Data_6_3                  CVD_Data_Protest              x         
CVD_Data_7_3                  CVD_Data_Protest              x         
CVD_Data_8_3                  CVD_Data_Protest              x         
CVD_Data_9_3                  CVD_Data_Protest              x         
CVD_Data_10_3                 CVD_Data_Protest              x         
CVD_Data_11_3                 CVD_Data_Protest              x         
CVD_Data_12_3                 CVD_Data_Protest              x         
CVD_Data_13_3                 CVD_Data_Protest              x         
CVD_Da

<ipython-input-238-26ec6d50ceb8>:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider using pd.concat instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  export(request, save_filename, data)


mismatch_dem_age


In [239]:
# 9. Marie Christine: Resilience, Self, and Mental Health: protective factors and risk factors in acute, then chronic stress
request_filename = r'C:\Users\andyl\Box\COVID-19 Adolphs Lab\approved_projects\Marie_Christine_Resilience\request.csv'
request_col = 2
save_filename = './output/approved_projects/Marie_Christine_Resilience/data.csv'
request = process_request(request_filename, request_col, save_filename)
export(request, save_filename, data)

UNV                           request_label                 x         

These columns were not found in the data:
mismatch_dem_age
['14', 'F', '8', '3', 'N', '15', 'K', '5', 'I', 'L', '13', 'M', 'J', '15b', 'E', 'C', '4', 'B', '7', 'G', 'A', '1', '9', '10', '16', 'D', '17', '11', '6', '12', '2', 'H']
8
['14', 'F', '3', 'N', '15', 'K', '5', 'I', 'L', 'M', 'J', '15b', 'E', 'C', '4', 'B', '7', 'G', 'A', '1', '9', '10', '16', 'D', '11', '6', '12', '2', 'H']


<ipython-input-239-c5b432a89a65>:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider using pd.concat instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  export(request, save_filename, data)


mismatch_dem_age


In [240]:
# 10. Mike: The Impact of Psychological Traits, Political Attitudes, and COVID-Specific Concerns on Trust and Compliance in the COVID-19 Pandemic
request_filename = r'C:\Users\andyl\Box\COVID-19 Adolphs Lab\approved_projects\Mike_Alverez_The_Impact_of_Psychological_Traits_Political_Attitudes_and_COVID-Specific_Concerns\request.csv'
request_col = 2
save_filename = './output/approved_projects/Mike_Alverez_The_Impact_of_Psychological_Traits_Political_Attitudes_and_COVID-Specific_Concerns/data.csv'
request = process_request(request_filename, request_col, save_filename)
export(request, save_filename, data)

UNV                           request_label                 x         
CVD_Data_1_4                  Unemployment_Stats            x         
CVD_Data_2_4                  Unemployment_Stats            x         

These columns were not found in the data:
state
cumulative_cases
new_cases
slope_new_cases
cumulative_deaths
new_deaths
slope_new_deaths
Mandatory_SAH
Recomm_SAH
Mandatory_gather_limit
Mandatory_gather_max
Recomm_gather_limit
Recomm_gather_max
GatheringStrictness
State_park_closure
Playground_closure
Mandatory_PPE_masks
Recomm_PPE_masks
Mandatory_business_closure
Recomm_business_closure
mismatch_dem_age
['14', 'F', '8', '3', 'N', '15', 'K', '5', 'I', 'L', '13', 'M', 'J', '15b', 'E', 'C', '4', 'B', '7', 'G', 'A', '1', '9', '10', '16', 'D', '17', '11', '6', '12', '2', 'H']


<ipython-input-240-2be4b09fbd7e>:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider using pd.concat instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  export(request, save_filename, data)


mismatch_dem_age


In [241]:
# 11. Yue: Evaluation of metrics for identifying and distinguishing psychological states from traits using COVID-dynamic dataset
request_filename = r'C:\Users\andyl\Box\COVID-19 Adolphs Lab\approved_projects\Yue_Xu_Evaluation_of_metrics_for_identifying_and_distinguishing_psychological_states_from_traits\request.csv'
request_col = 2
save_filename = './output/approved_projects/Yue_Xu_Evaluation_of_metrics_for_identifying_and_distinguishing_psychological_states_from_traits/data.csv'
request = process_request(request_filename, request_col, save_filename)
export(request, save_filename, data)

UNV                           request_label                 x         
EMS_P_2_1                     EMSC                          x         1.0       
EMS_P_2_2                     EMSC                          x         1.0       
EMS_P_4_1                     EMSC                          x         3.0       
EMS_P_4_2                     EMSC                          x         3.0       
EMS_P_5_1                     EMSC                          x         3.0       
EMS_P_5_2                     EMSC                          x         3.0       
EMS_P_10_1                    EMSC                          x         9.0       
EMS_P_10_2                    EMSC                          x         9.0       

These columns were not found in the data:
mismatch_dem_age
['14', 'F', '8', '3', 'N', '15', 'K', '5', 'I', 'L', '13', 'M', 'J', '15b', 'E', 'C', '4', 'B', '7', 'G', 'A', '1', '9', '10', '16', 'D', '17', '11', '6', '12', '2', 'H']


<ipython-input-241-ec174c11638b>:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider using pd.concat instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  export(request, save_filename, data)


mismatch_dem_age


In [242]:
# 12. Yanting: Emotion Projects
request_filename = r'C:\Users\andyl\Box\COVID-19 Adolphs Lab\approved_projects\Yanting_Han_Emotion_Projects\request.csv'
request_col = 2
save_filename = './output/approved_projects/Yanting_Han_Emotion_Projects/data.csv'
request = process_request(request_filename, request_col, save_filename)
export(request, save_filename, data)

UNV                           request_label                 x         
CVD_Data_1_3                  CVD_Data_Protest              x         
CVD_Data_1_4                  Unemployment_Stats            x         
CVD_Data_2_3                  CVD_Data_Protest              x         
CVD_Data_2_4                  Unemployment_Stats            x         
CVD_Data_3_3                  CVD_Data_Protest              x         
CVD_Data_3_4                  Unemployment_Stats            x         
CVD_Data_4_3                  CVD_Data_Protest              x         
CVD_Data_4_4                  Unemployment_Stats            x         
CVD_Data_5_3                  CVD_Data_Protest              x         
CVD_Data_5_4                  Unemployment_Stats            x         
CVD_Data_6_3                  CVD_Data_Protest              x         
CVD_Data_6_4                  Unemployment_Stats            x         
CVD_Data_7_3                  CVD_Data_Protest              x         
CVD_Da

<ipython-input-242-fe170184991e>:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider using pd.concat instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  export(request, save_filename, data)


mismatch_dem_age


In [243]:
# 13. Lynn: NDAR Data Submission for Conte
request_filename = r'C:\Users\andyl\Box\COVID-19 Adolphs Lab\approved_projects\Lynn_Paul_NDAR_Data_Submission_for_Conte\request.csv'
request_col = 2
save_filename = './output/approved_projects/Lynn_Paul_NDAR_Data_Submission_for_Conte/data.csv'
request = process_request(request_filename, request_col, save_filename)
export(request, save_filename, data)

UNV                           request_label                 x         
DSR_C_1                       nan                           x         
EmoSp_C_2                     EmoSp                         x         nan       
EmoSp_C_4                     EmoSp                         x         nan       
EMS_C_2_1                     EMSC                          x         A         
EMS_C_2_2                     EMSC                          x         A         
EMS_C_4_1                     EMSC                          x         B         
EMS_C_4_2                     EMSC                          x         B         
EMS_C_5_1                     EMSC                          x         B         
EMS_C_5_2                     EMSC                          x         B         
EMS_C_10_1                    EMSC                          x         F         
EMS_C_10_2                    EMSC                          x         F         
H_E_C_1                       H_E               

<ipython-input-243-c717968264a5>:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider using pd.concat instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  export(request, save_filename, data)


mismatch_dem_age


In [244]:
# 14 Marie Christine: Self under stress
request_filename = r'C:\Users\andyl\Box\COVID-19 Adolphs Lab\approved_projects\Marie_Christine_The_Self_under_Stress\request.csv'
request_col = 2
save_filename = './output/approved_projects/Marie_Christine_The_Self_under_Stress/data.csv'
request = process_request(request_filename, request_col, save_filename)
export(request, save_filename, data)

UNV                           request_label                 x         
CVD_Data_1_3                  CVD_Data_Protest              x         
CVD_Data_1_4                  Unemployment_Stats            x         
CVD_Data_2_3                  CVD_Data_Protest              x         
CVD_Data_2_4                  Unemployment_Stats            x         
CVD_Data_3_3                  CVD_Data_Protest              x         
CVD_Data_3_4                  Unemployment_Stats            x         
CVD_Data_4_3                  CVD_Data_Protest              x         
CVD_Data_4_4                  Unemployment_Stats            x         
CVD_Data_5_3                  CVD_Data_Protest              x         
CVD_Data_5_4                  Unemployment_Stats            x         
CVD_Data_6_3                  CVD_Data_Protest              x         
CVD_Data_6_4                  Unemployment_Stats            x         
CVD_Data_7_3                  CVD_Data_Protest              x         
CVD_Da

<ipython-input-244-d9222eee9645>:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider using pd.concat instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  export(request, save_filename, data)


mismatch_dem_age


In [245]:
# 15 Tessa: Self Covid Core
request_filename = r'C:\Users\andyl\Box\COVID-19 Adolphs Lab\approved_projects\Tessa_Covid_Core\request.csv'
request_col = 3
save_filename = './output/approved_projects/Tessa_Covid_Core/data.csv'
request = process_request(request_filename, request_col, save_filename)
export(request, save_filename, data)

UNV                           request_label                 x         
CVD_Data_1_3                  CVD_Data_Protest              x         
CVD_Data_1_4                  Unemployment_Stats            x         
CVD_Data_2_3                  CVD_Data_Protest              x         
CVD_Data_2_4                  Unemployment_Stats            x         
CVD_Data_3_3                  CVD_Data_Protest              x         
CVD_Data_3_4                  Unemployment_Stats            x         
CVD_Data_4_3                  CVD_Data_Protest              x         
CVD_Data_4_4                  Unemployment_Stats            x         
CVD_Data_5_3                  CVD_Data_Protest              x         
CVD_Data_5_4                  Unemployment_Stats            x         
CVD_Data_6_3                  CVD_Data_Protest              x         
CVD_Data_6_4                  Unemployment_Stats            x         
CVD_Data_7_3                  CVD_Data_Protest              x         
CVD_Da

<ipython-input-245-b4b007cf7949>:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider using pd.concat instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  export(request, save_filename, data)


mismatch_dem_age


In [246]:
# 15 Andy: Causal Effect of Lockdown
request_filename = r'C:\Users\andyl\Box\COVID-19 Adolphs Lab\approved_projects\Andy_Causal_Effect_of_Lockdown\request.csv'
request_col = 2
save_filename = './output/approved_projects/Andy_Causal_Effect_of_Lockdown/data.csv'
request = process_request(request_filename, request_col, save_filename)
export(request, save_filename, data)

UNV                           request_label                 x         
DSR_C_1                       nan                           x         
NIH_C_1                       NIH                           x         nan       
NIH_C_3                       NIH                           x         nan       
PSS_C_2                       PSS                           x         nan       
PSS_C_4                       PSS                           x         nan       

These columns were not found in the data:
mismatch_dem_age
['14', 'F', '8', '3', 'N', '15', 'K', '5', 'I', 'L', '13', 'M', 'J', '15b', 'E', 'C', '4', 'B', '7', 'G', 'A', '1', '9', '10', '16', 'D', '17', '11', '6', '12', '2', 'H']


<ipython-input-246-3bd608a649e1>:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider using pd.concat instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  export(request, save_filename, data)


mismatch_dem_age


In [225]:
request_filename = r'C:\Users\andyl\Box\COVID-19 Adolphs Lab\approved_projects\Andy_Liang_Uncovering_Heterogeneity_within_Conspiracy_Theorists\request.csv'
request_col = 2
request = inspect_request(request_filename)
request

,index,0,1
17,Invest_2,"Prior to submitting a data-request, the COVID-...",Uncovering Heterogeneity within Conspiracy The...
18,Invest_3,"Prior to submitting a data-request, the COVID-...",Andy Liang
19,Invest_4,"Prior to submitting a data-request, the COVID-...",Chapman University
20,Invest_5,"Prior to submitting a data-request, the COVID-...",liang134@mail.chapman.edu
21,Other_Investigators#1_1_1,All data will be distributed via Box to the PI...,NaN
...,...,...,...
1561,GFPS36v2_2,"Protest Survey 2 - Wave 8 - June 27, 2020 (Sel...",NaN
1562,GFPS37v2_1,"Protest Survey 2 - Wave 8 - June 27, 2020 (Sel...",NaN
1563,GFPS37v2_2,"Protest Survey 2 - Wave 8 - June 27, 2020 (Sel...",NaN
1564,GFPS38v2_1,"Protest Survey 2 - Wave 8 - June 27, 2020 (Sel...",NaN


In [180]:
request.loc[request['index'] == 'MHC_P_17']

,index,0,1
500,MHC_P_17,Mental Health Checkpoint - Prolific - Wave 17-...,NaN


In [181]:
request_filename = '/Users/Work/Box/COVID-19 Adolphs Lab/approved_projects/Andy_Liang_Uncovering_Heterogeneity_within_Conspiracy_Theorists/request.csv'